In [ ]:
import rasterio
import os
from datetime import datetime
import numpy as np

In [ ]:
# local libraries
from coupling_PCR_FM.model_functions_v2 import PCR_model, CMF_model, DFM_model
from coupling_PCR_FM.utils import config_to_dict
%config Application.log_level="INFO"

In [ ]:
# IMPORT MODEL SETTINGS FROM INI-FILE
argv1 = r'default_rename.set'
argv2 = r'paths.env'

# parse set/ini-file with central/general settings for coupling framework
config = config_to_dict(argv1)
# parse env-file for user-specific paths and environmental variables
envs = config_to_dict(argv2)
# combine
config.update(envs)
options = config
# parse dates
start_date = datetime.strptime(options['numerical_settings']['startTime'], '%Y-%m-%d')
end_date = datetime.strptime(options['numerical_settings']['endTime'], '%Y-%m-%d')

In [ ]:
# setup output dir
cwd = os.getcwd() # note: this get changed by pcr initialization later on
out_dir = options['PCRpaths']['outputDirectoryPCR']
if not os.path.isdir(out_dir):
    os.mkdir(out_dir)
print(out_dir)
print(cwd)

## create PCR model BMI object

In [ ]:
PCR_configfile = os.path.join(cwd, options['hydrologic_model']['config_dir'], options['hydrologic_model']['config_file'])
PCR_in_dir = options['PCRpaths']['inputDirectoryPCR']
PCR_out_dir = os.path.join(out_dir, 'PCR')

PCR_bmi = PCR_model(PCR_configfile, PCR_in_dir, PCR_out_dir,
                            start_date, end_date, dt=1)


## create CMF model BMi object

In [ ]:
CMF_engine = os.path.join(cwd, options['CMF_engine']['CMF_path'])
CMF_model_dir = os.path.join(cwd, options['routing_model']['model_dir'])
CMF_config_fn = os.path.join(CMF_model_dir, options['routing_model']['model_file'])
CMF_out_dir = os.path.join(out_dir, 'CMF')
     
CMF_bmi = CMF_model(CMF_engine, CMF_config_fn, CMF_model_dir, CMF_out_dir,
                         start_date, end_date, dt=86400)




## initialize upstream hydrological model to expose LDD via BMI

In [ ]:
# initialize model at PCR_out_dir
PCR_bmi.initialize()

## set coupling

In [ ]:
# get model grid defition and index
PCR_bmi.get_model_2d_index()

# set inpmat based on upstream hydrological model
bounds, res = PCR_bmi.model_grid_bounds, PCR_bmi.model_grid_res
CMF_bmi.set_inpmat_file(bounds, res)

# reset PCR ldd -> no routing
PCR_bmi.deactivate_LDD('all')

In [ ]:
# set update states function
def PCR2CMF_runoff(PCR_bmi, CMF_bmi):
    "creates a 'get_runoff' function to get runoff from PCR model fit for CMF model"
    def update_states():
        "coupling runoff between CMFan PCR model"
        runoff = PCR_bmi.get_var('landSurfaceRunoff')
        runoff = np.where(np.isnan(runoff), 0, runoff)
        CMF_bmi.set_var("runoff", runoff)
    return update_states

update_states = PCR2CMF_runoff(PCR_bmi, CMF_bmi)

## initialize downstream routing model

In [ ]:
# initialize model at CMF_out_dir
CMF_bmi.initialize()

## run coupled PCR - CMF model

In [ ]:
# run coupled model for 20 timesteps
for i in range(20):
    PCR_bmi.update()
    update_states()
    CMF_bmi.update()   

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

# PCR_bmi.update()
# routingModel.update_states()
# routingModel.update() 

runoff = PCR_bmi.get_var('landSurfaceRunoff')
outflw = CMF_bmi.get_var('outflw')
outflw = np.ma.masked_greater(outflw, 1.e19)
rivsto = CMF_bmi.get_var('rivsto_out')
rivsto = np.ma.masked_greater(rivsto, 1.e19)

fig = plt.figure(figsize=(15, 5))
a=fig.add_subplot(1,3,1)
imgplot = plt.imshow(runoff)
a.set_title('PCR runoff')
plt.colorbar(orientation ='horizontal')
a=fig.add_subplot(1,3,2)
imgplot = plt.imshow(outflw)
a.set_title('CMF outflw')
plt.colorbar(orientation='horizontal')
a=fig.add_subplot(1,3,3)
imgplot = plt.imshow(rivsto)
a.set_title('CMF rivsto')
plt.colorbar(orientation='horizontal')

In [ ]:
# finalize model
PCR_bmi.finalize()

# finalize model
CMF_bmi.finalize()